In [ ]:
from utils.data_preprocess import load_single_leakage_model_data
from utils.model_evaluation import plot_test_pred
from utils.module import model_eval
import yaml
import tensorflow as tf
from tensorflow import keras
from keras import layers
import keras_tuner as kt
import itertools
import numpy as np
 
with open("config.yml", "r") as ymlfile:
    cfg = yaml.full_load(ymlfile)

In [ ]:
residual_subtract = False
augmentation = False
blind_flip = False
tot_flow = False
tot_resflow = False
res_flow = False

In [ ]:
# model = tf.keras.models.load_model('saved_model/single_leakage_model_new')
# model.summary()

In [ ]:
X_train, X_test, X_val, y_train, y_test, y_val, scaler_coords, scaler_flows  = load_single_leakage_model_data(residual_subtract, augmentation, 
                                                                                                    blind_flip, tot_flow, 
                                                                                                    res_flow, tot_resflow)

In [ ]:
import pandas as pd
results = pd.DataFrame(columns=['start_learning_rate', 'width', 'depth', 'l2_weight', 'train_loss', 'val_loss', 'train_acc', 'val_acc'])

epoch = 1000
start_learning_rates = [1e-3, 1e-2, 1e-4]
widths_low = [2, 4, 8, 16]
widths_high = list(range(32, 512, 32))
widths = widths_high + widths_low
depths = list(range(1, 16,1))
l2_weights = [0, 1e-1, 1e-2, 1e-3]
l1_weights = [0, 1e-1, 1e-2, 1e-3]

end_learning_rate = 1e-8
decay_steps = 10000
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)

for start_learning_rate in start_learning_rates:
    for width in widths:
        for depth in depths:
            for l2_weight in l2_weights:
                for l1_weight in l1_weights:
                    model = keras.models.Sequential()
                    for _ in range(depth):
                        model.add(keras.layers.Dense(units=width, activation="relu", 
                                                     kernel_regularizer=keras.regularizers.L1L2(l1 = l1_weight, l2 = l2_weight),
                                                     kernel_initializer='he_uniform'))
                    model.add(tf.keras.layers.Dense(2,activation= "linear", kernel_initializer='he_uniform'))
                    scheduler = tf.keras.optimizers.schedules.PolynomialDecay(initial_learning_rate= start_learning_rate,
                                                                              decay_steps= decay_steps, 
                                                                              end_learning_rate= end_learning_rate,
                                                                              power=1)
                    model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=scheduler),
                                  loss="mse",
                                  metrics='mae')
                    history = model.fit(X_train, y_train, epochs=epoch, 
                                        validation_data = (X_val, y_val), 
                                        callbacks= stop_early,
                                        shuffle= True)
                    train_loss, train_acc = model.evaluate(X_train, y_train)
                    val_loss, val_acc = model.evaluate(X_val, y_val)
                    results_tmp = np.array([start_learning_rate, width, depth, l2_weight, train_loss, val_loss, train_acc, val_acc]).reshape(1, -1)
                    results = results.append(pd.DataFrame(data=results_tmp, columns=results.columns), ignore_index=True)
results.to_csv('results.csv')

Epoch 423/1000
11/11 [==============================] - 0s 6ms/step - loss: 0.1161 - mae: 0.1202 - val_loss: 0.1098 - val_mae: 0.1080
Epoch 424/1000
11/11 [==============================] - 0s 6ms/step - loss: 0.1157 - mae: 0.1203 - val_loss: 0.1098 - val_mae: 0.1083
Epoch 425/1000
11/11 [==============================] - 0s 6ms/step - loss: 0.1157 - mae: 0.1204 - val_loss: 0.1098 - val_mae: 0.1084
Epoch 426/1000
11/11 [==============================] - 0s 6ms/step - loss: 0.1159 - mae: 0.1206 - val_loss: 0.1106 - val_mae: 0.1111
Epoch 427/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.1168 - mae: 0.1234 - val_loss: 0.1113 - val_mae: 0.1134
Epoch 428/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.1159 - mae: 0.1188 - val_loss: 0.1105 - val_mae: 0.1120
Epoch 429/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.1156 - mae: 0.1214 - val_loss: 0.1125 - val_mae: 0.1149
Epoch 430/1000
11/11 [==============================] - 0s 7ms

11/11 [==============================] - 0s 10ms/step - loss: 0.0335 - mae: 0.0848 - val_loss: 0.0318 - val_mae: 0.0847
Epoch 460/1000
11/11 [==============================] - 0s 10ms/step - loss: 0.0332 - mae: 0.0841 - val_loss: 0.0326 - val_mae: 0.0869
Epoch 461/1000
11/11 [==============================] - 0s 9ms/step - loss: 0.0347 - mae: 0.0889 - val_loss: 0.0338 - val_mae: 0.0900
Epoch 462/1000
11/11 [==============================] - 0s 9ms/step - loss: 0.0345 - mae: 0.0897 - val_loss: 0.0333 - val_mae: 0.0899
Epoch 463/1000
11/11 [==============================] - 0s 8ms/step - loss: 0.0335 - mae: 0.0858 - val_loss: 0.0320 - val_mae: 0.0827
Epoch 464/1000
11/11 [==============================] - 0s 11ms/step - loss: 0.0332 - mae: 0.0845 - val_loss: 0.0318 - val_mae: 0.0835
Epoch 465/1000
11/11 [==============================] - 0s 9ms/step - loss: 0.0329 - mae: 0.0833 - val_loss: 0.0318 - val_mae: 0.0840
Epoch 466/1000
11/11 [==============================] - 0s 8ms/step - loss

In [ ]:
model_evaluate, y_pred = model_eval(model, X_test, y_test, X_train, y_train, X_val, y_val)
plot_test_pred(y_test, y_pred, scaler_coords)